In [23]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [29]:
merged = pd.read_csv(path+'data/merged.csv')

test = merged.copy()

In [25]:
test = merged.drop(['country', 'lat', 'long', 'date', 'date_of_birth', 'nationality', 'driver_wins_after_race', 'driver_standings_pos_after_race',  'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis=1)

In [30]:
driver_points_per_race = []
driver_points_from_race = []

constructor_points_per_race = []
constructor_points_from_race = []

for index, row in test.iterrows():
    driver_points_per_race.append(round(row.driver_points / row['round'], 2))
    driver_points_from_race.append(round(row.driver_points_after_race - row.driver_points, 1))

    constructor_points_per_race.append(round(row.driver_points / row['round'], 2))
    constructor_points_from_race.append(round(row.constructor_points_after_race - row.constructor_points, 1))

In [31]:
# test['driver_points_per_race'] = driver_points_per_race
test['driver_points_from_race'] = driver_points_from_race

# test['constructor_points_per_race'] = constructor_points_per_race
test['constructor_points_from_race'] = constructor_points_from_race

In [32]:
test.query('season == 2014 & round == 1')

,season,round,circuit_id,country,lat,long,date,driver,date_of_birth,nationality,...,constructor_points,constructor_wins,constructor_standings_pos,qualifying_pos,qual_time,stage,q_delta,starting_grid,driver_points_from_race,constructor_points_from_race
0,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,nico_rosberg,1985-06-27,German,...,0,0,0,3,104.595,q3,0.000,3,25.0,25.0
1,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,kevin_magnussen,1992-10-05,Danish,...,0,0,0,4,105.745,q3,-6.088,4,18.0,33.0
2,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,jenson_button,1980-01-19,British,...,0,0,0,11,104.437,q2,0.206,10,15.0,33.0
3,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,fernando_alonso,1981-07-29,Spanish,...,0,0,0,5,105.819,q3,-6.014,5,12.0,18.0
4,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,valtteri_bottas,1989-08-28,Finnish,...,0,0,0,10,108.147,q3,3.916,15,10.0,10.0
5,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,nico_hulkenberg,1987-08-19,German,...,0,0,0,7,106.030,q3,-5.803,7,8.0,9.0
6,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,kimi_raikkonen,1979-10-17,Finnish,...,0,0,0,12,104.494,q2,0.263,11,6.0,18.0
7,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,jean-eric_vergne,1990-04-25,French,...,0,0,0,6,105.864,q3,-5.969,6,4.0,6.0
8,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,daniil_kvyat,1994-04-26,Russian,...,0,0,0,8,107.368,q3,-4.465,8,2.0,6.0
9,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16 06:00:00,sergio_perez,1990-01-26,Mexican,...,0,0,0,16,107.293,q2,3.062,16,1.0,9.0


In [80]:
round = 7
list(range(round-1, round-4, -1))

[6, 5, 4]

In [100]:
grouped = test.groupby(['season', 'round'])
last_3_races = []

for index, group in grouped:
        for _, row in group.iterrows():
                round = index[1]
                rounds = list(range(round-1, round-4, -1))
                points = test.query(
                        'season ==@index[0] & round in @rounds & driver ==@row.driver'
                        ).agg({
                                'driver_points_from_race': sum
                                }).values[0]

                last_3_races.append(points)

In [97]:
grouped = test.groupby(['season', 'round'])
points_before_race = []

for index, group in grouped:
        for _, row in group.iterrows():
                round = index[1]
                query = test.query(
                        'season ==@index[0] & round ==@round-1 & driver ==@row.driver')

                points = query.driver_points_after_race
                if len(points) != 0:
                        points_before_race.append(points.values[0])
                else:
                        points_before_race.append(0.0)

In [101]:
test['driver_points_last_3_races'] = last_3_races
test['driver_points_before_race'] = points_before_race

In [107]:
testing = test[['season','round', 'driver', 'driver_points',  'driver_points_before_race', 'driver_points_from_race', 'driver_points_last_3_races']]

In [110]:
testing.query('season == 2021 & driver == "max_verstappen"')

,season,round,driver,driver_points,driver_points_before_race,driver_points_from_race,driver_points_last_3_races
2761,2021,1,max_verstappen,0,0.0,18.0,0.0
2780,2021,2,max_verstappen,18,18.0,25.0,18.0
2800,2021,3,max_verstappen,43,43.0,18.0,43.0
2820,2021,4,max_verstappen,61,61.0,19.0,61.0
2839,2021,5,max_verstappen,80,80.0,25.0,62.0
2874,2021,6,max_verstappen,105,105.0,0.0,62.0
2876,2021,7,max_verstappen,105,105.0,26.0,44.0
2894,2021,8,max_verstappen,131,131.0,25.0,51.0
2914,2021,9,max_verstappen,156,156.0,26.0,51.0
2953,2021,10,max_verstappen,182,182.0,3.0,77.0


In [8]:
merged.columns

Index(['season', 'round', 'circuit_id', 'country', 'lat', 'long', 'date',
       'driver', 'date_of_birth', 'nationality', 'constructor', 'podium',
       'driver_points_after_race', 'driver_wins_after_race',
       'driver_standings_pos_after_race', 'driver_points', 'driver_wins',
       'driver_standings_pos', 'constructor_points_after_race',
       'constructor_wins_after_race', 'constructor_standings_pos_after_race',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'qualifying_pos', 'qual_time', 'stage', 'q_delta', 'starting_grid'],
      dtype='object')

In [16]:
merged[['driver', 'constructor', 
        # 'podium',
    #    'driver_points_after_race', 'driver_wins_after_race',
    #    'driver_standings_pos_after_race', 'constructor_points_after_race',
    #    'constructor_wins_after_race', 'constructor_standings_pos_after_race',
    #    'qualifying_pos',
        # 'starting_grid',
    #    'qual_time', 
       'stage', 'q_delta']].groupby('stage').describe()

q_delta                                                             
        count      mean       std     min      25%     50%     75%     max
stage                                                                     
q1      926.0  3.401348  3.943653 -17.098  2.38150  3.0690  3.9105  33.846
q2      932.0  2.108410  2.250302 -15.988  1.57675  2.1005  2.5380  26.141
q3     1761.0  1.216859  1.614988  -6.088  0.32000  0.9150  1.5680  22.112

In [34]:
merged.to_csv(path+'data/merged.csv', index=False)